In [1]:
import os
import numpy as np

from silx.io.dictdump import h5todict, dicttoh5

In [2]:
cm3_to_m3 = 1e-6

Inputs

In [3]:
nb_states = 6115
kin_file = "/home/zanardi/Workspace/AirDatabase/HDF5/QCT/Orig/O3_UMN.hdf5"

In [4]:
# Save path
out_path = "./../"
os.makedirs(out_path, exist_ok=True)

Kinetics database

In [5]:
kin_dtb = {}
kinetics = h5todict(kin_file)

> Molecule - Atom collisions

In [6]:
ma_dtb = kinetics["T_10000_10000"]["Rates"]

In [7]:
kin_dtb["m-a"] = {}
for (k, rates) in ma_dtb.items():
  if (k == "Diss_Corr"):
    p = "d"
    shape = (nb_states,1,1,1,1)
  elif (k == "Excit"):
    p = "e"
    shape = (nb_states,1,nb_states,1)
    # Remove endothermic processes
    triu_i = np.triu_indices(nb_states)
    rates[triu_i[0],triu_i[1]] = 0.0
  else:
    continue
  # Reshape and rescale
  rates = rates.reshape(shape) * cm3_to_m3
  # Store
  kin_dtb["m-a"][p] = rates

In [8]:
# Set Arrhenius parameters
for (p, rates) in kin_dtb["m-a"].items():
  zeros = np.zeros_like(rates)
  kin_dtb["m-a"][p] = {"A": rates, "beta": zeros, "Ta": zeros}

> Save database

In [9]:
dicttoh5(kin_dtb, out_path + "/kinetics.hdf5")